In [ ]:
import torch
import os
import sys
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
sys.path.append(r"E:\Projects\Deep_Learning\Car_Segmentation\Utils")

from unet_model import UNET
from segmentation_utils import SegmentationDataset, load_checkpoint, view_model

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = UNET(in_channels=3, out_channels=5).to(DEVICE)

In [ ]:
if os.path.exists("../Model Checkpoints/car_segmentation_diceceloss_checkpoint.pth.tar"):
        load_checkpoint(torch.load("../Model Checkpoints/car_segmentation_diceceloss_checkpoint.pth.tar"), model)
        print("=> Checkpoint Loaded")

In [ ]:
ds = SegmentationDataset(dirPath="../Data/car_dataset", imageDir='images/', masksDir='masks/', img_size=512)
car_dataloader = DataLoader(ds, 2, shuffle=True, num_workers=0, pin_memory=False)

In [ ]:
def dice_coef(groundtruth_mask, pred_mask):
        print("groundtruth_mask.shape =", groundtruth_mask.shape, "pred_mask.shape = ", pred_mask.shape)
        if groundtruth_mask.shape != pred_mask.shape:
                raise ValueError("Shape mismatch: groundtruth_mask and pred_mask must have to be of the same shape.")
        intersect = np.sum(pred_mask*groundtruth_mask)
        total_sum = np.sum(pred_mask) + np.sum(groundtruth_mask)
        dice = np.mean(2*intersect/total_sum)
        return round(dice, 3) #round up to 3 decimal places

In [ ]:
for i in car_dataloader:
    img , gt_mask = i
    print("gt = ",gt_mask.shape)
    
    img = img.to(DEVICE)
    print("img = ", img.shape)
    print("img new = ", img[0].shape, img[0].permute(1,2,0).shape)
    with torch.no_grad():
        out = model(img)
    
    raw = out
    print("raw = ",out.shape)
    
    probs = torch.nn.functional.softmax(out,dim=1)
    raw = torch.nn.functional.softmax(raw,dim=1)
    
    out = torch.argmax(probs, dim=1,keepdim=True)
    
    print("out = ", out.shape)
    
    mask = out.detach().cpu()
    
    print("mask.shape  =", mask.shape)
    print("gt = ", gt_mask.shape)
    
    plt.subplot(1,3,1)
    plt.imshow(img[0].permute(1,2,0).cpu())
    plt.title('Image')
    plt.axis(False)
    plt.subplot(1,3,2)
    plt.title('PREDS')
    # plt.imshow(raw[0][2].cpu(), cmap='copper')
    plt.imshow(mask[0].permute(1,2,0).cpu(), cmap='copper')
    plt.axis(False)
    plt.subplot(1,3,3)
    plt.title('Mask')
    plt.axis(False)
    plt.imshow(gt_mask[0][2], cmap='gray')
    plt.show()
    break

In [ ]:
model = view_model("E:/Projects/Deep_Learning/Car_Segmentation//ModelCheckpoints/Car_Segmentation_Unet_xception_CrossEntropyLoss_1/")